In [1]:
import pandas as pd

### Importo los datos

In [2]:
df_comisarias = pd.read_parquet(r'D:\0_Respaldo\0_Proyectos_2024\Henry_Labs\Lab2\Baires_Accidents\Data\Parquet\comisarias.parquet')

In [3]:
df_comuna = pd.read_parquet(r'D:\0_Respaldo\0_Proyectos_2024\Henry_Labs\Lab2\Baires_Accidents\Data\Parquet\comuna.parquet')

In [4]:
df_hechos = pd.read_parquet(r'D:\0_Respaldo\0_Proyectos_2024\Henry_Labs\Lab2\Baires_Accidents\Data\Parquet\hechos.parquet')

In [5]:
df_poblacion = pd.read_parquet(r'D:\0_Respaldo\0_Proyectos_2024\Henry_Labs\Lab2\Baires_Accidents\Data\Parquet\poblacion.parquet')

### Objetivo
1/ Crear un dataset que contenga los accidentes por comunas generales en el periodo de tiempo del 2010 al 2021.

2/ Crear un segundo dataset que contenga los accidentes registrados. el tipo de accidente la ubicacion geografica y la estacion de policia mas cercana.

### Tipos de accidentes por comuna para los anios 2020 y 2021

In [6]:
df_poblacion.head(3)

,COMUNA,2021,2020
0,0,3078836,3075646
1,1,257235,256405
2,2,149371,149430


In [7]:
df_hechos.head(3)

,ID,N_VICTIMAS,AAAA,COMUNA,Longitud,Latitud,VICTIMA
0,2016-0001,1.0,2016,8,-58.475340,-34.687570,MOTO
1,2016-0002,1.0,2016,9,-58.508775,-34.669777,AUTO
2,2016-0003,1.0,2016,1,-58.390403,-34.631894,MOTO


In [85]:

#Copio el dataframe y las columnas con que voy a trabajar y filtro para los anios a partir del 2020
df1 = df_hechos[df_hechos['AAAA'] >= 2020].copy()

In [86]:
df1

,ID,N_VICTIMAS,AAAA,COMUNA,Longitud,Latitud,VICTIMA
512,2020-0001,1.0,2020,15,-58.439104,-34.592134,PEATON
513,2020-0002,1.0,2020,7,-58.450194,-34.646464,MOTO
514,2020-0003,1.0,2020,3,-58.393376,-34.609276,PEATON
515,2020-0004,1.0,2020,1,-58.390556,-34.609193,PEATON
516,2020-0005,1.0,2020,3,-58.410370,-34.624770,MOTO
...,...,...,...,...,...,...,...
679,2021-0093,1.0,2021,7,-58.433538,-34.645616,MOTO
680,2021-0094,1.0,2021,9,-58.467398,-34.651178,MOTO
681,2021-0095,1.0,2021,11,-58.472934,-34.619847,MOTO
682,2021-0096,1.0,2021,9,-58.470668,-34.650217,AUTO


In [87]:
df1.to_parquet(r'D:\0_Respaldo\0_Proyectos_2024\Henry_Labs\Lab2\Baires_Accidents\Data\ETL\siniestros.parquet')

### Accidentes totales por comuna

In [19]:
df2 = df1.drop(columns=['VICTIMA'])

In [20]:
df2.sample(3)

,N_VICTIMAS,AAAA,COMUNA
656,1.0,2021,8
546,1.0,2020,14
640,1.0,2021,4


In [26]:
df2_dummy = pd.get_dummies(df2['AAAA'], dtype=int)

In [27]:
df2_dummy

,2020,2021
512,1,0
513,1,0
514,1,0
515,1,0
516,1,0
...,...,...
679,0,1
680,0,1
681,0,1
682,0,1


In [28]:
df3 = pd.concat([df2, df2_dummy], axis=1)

In [29]:
df3.sample(3)

,N_VICTIMAS,AAAA,COMUNA,2020,2021
577,1.0,2020,1,1,0
632,1.0,2021,15,0,1
588,1.0,2021,1,0,1


In [31]:
df3.columns

Index(['N_VICTIMAS', 'AAAA', 'COMUNA', 2020, 2021], dtype='object')

In [52]:
df4 = df3.groupby(['COMUNA', 2020, 2021])['N_VICTIMAS'].sum().reset_index()

In [53]:
df4.head()

,COMUNA,2020,2021,N_VICTIMAS
0,1,0,1,9.0
1,1,1,0,10.0
2,2,0,1,4.0
3,2,1,0,5.0
4,3,0,1,8.0


In [49]:
#Creo la funcion para copiar los valores de las victimas bajo su respectivo anio.

def copy_to_year(row):
    if row[2020] == 1:
        row[2020] = row['N_VICTIMAS']
    elif row[2021] == 1:
        row[2021] = row['N_VICTIMAS']
    return row


In [54]:
# Modifico el df con la funcion creada.
df4[[2020, 2021]] = df4.apply(copy_to_year, axis=1)[[2020, 2021]]

In [55]:
df4

,COMUNA,2020,2021,N_VICTIMAS
0,1,0.0,9.0,9.0
1,1,10.0,0.0,10.0
2,2,0.0,4.0,4.0
3,2,5.0,0.0,5.0
4,3,0.0,8.0,8.0
5,3,10.0,0.0,10.0
6,4,0.0,12.0,12.0
7,4,4.0,0.0,4.0
8,5,0.0,4.0,4.0
9,5,3.0,0.0,3.0


#### Modifico el nombre de las columnas y elimino la columna 'N_VICTMIMA'

In [56]:
df4 =df4.rename(columns={2020:'Total victimas 2020'})

In [57]:
df4 =df4.rename(columns={2021:'Total victimas 2021'})

In [58]:
df4 = df4.drop(columns='N_VICTIMAS', axis=1)
df4

,COMUNA,Total victimas 2020,Total victimas 2021
0,1,0.0,9.0
1,1,10.0,0.0
2,2,0.0,4.0
3,2,5.0,0.0
4,3,0.0,8.0
5,3,10.0,0.0
6,4,0.0,12.0
7,4,4.0,0.0
8,5,0.0,4.0
9,5,3.0,0.0


In [76]:
# Agrupo nuevamente por comuna
df4 = df4.groupby('COMUNA')[['Total victimas 2020', 'Total victimas 2021']].sum().reset_index()

In [79]:
df4.head(3)

,COMUNA,Total victimas 2020,Total victimas 2021
0,1,10.0,9.0
1,2,5.0,4.0
2,3,10.0,8.0


#### Unir 'merge' los dataframes en la columna 'COMUNA'

In [63]:
df5 = pd.merge(df_poblacion, df4, on='COMUNA', how='inner')

In [64]:
df5

,COMUNA,2021,2020,Total victimas 2020,Total victimas 2021
0,1,257235,256405,10.0,9.0
1,2,149371,149430,5.0,4.0
2,3,193425,193276,10.0,8.0
3,4,240437,240100,4.0,12.0
4,5,187670,187518,3.0,4.0
5,6,185620,185456,2.0,5.0
6,7,242188,241861,6.0,6.0
7,8,229541,228953,7.0,7.0
8,9,171444,171264,6.0,11.0
9,10,170677,170592,4.0,2.0


In [80]:
# cambio el nombre de 2021 y 2020 a poblacion_2021 y 2020
df5 = df5.rename(columns={'2021':'Poblacion 2021'})

In [81]:
df5 = df5.rename(columns={'2020':'Poblacion 2020'})

In [82]:
df5.head(3)

,COMUNA,Poblacion 2021,Poblacion 2020,Total victimas 2020,Total victimas 2021
0,1,257235,256405,10.0,9.0
1,2,149371,149430,5.0,4.0
2,3,193425,193276,10.0,8.0


#### Verifico el dataframe

In [65]:
dfh = df_hechos.copy()

In [66]:
dfh

,ID,N_VICTIMAS,AAAA,COMUNA,Longitud,Latitud,VICTIMA
0,2016-0001,1.0,2016,8,-58.475340,-34.687570,MOTO
1,2016-0002,1.0,2016,9,-58.508775,-34.669777,AUTO
2,2016-0003,1.0,2016,1,-58.390403,-34.631894,MOTO
3,2016-0004,1.0,2016,8,-58.465039,-34.680930,MOTO
4,2016-0005,1.0,2016,1,-58.387183,-34.622466,MOTO
...,...,...,...,...,...,...,...
679,2021-0093,1.0,2021,7,-58.433538,-34.645616,MOTO
680,2021-0094,1.0,2021,9,-58.467398,-34.651178,MOTO
681,2021-0095,1.0,2021,11,-58.472934,-34.619847,MOTO
682,2021-0096,1.0,2021,9,-58.470668,-34.650217,AUTO


In [72]:
dfh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 684 entries, 0 to 683
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          684 non-null    object 
 1   N_VICTIMAS  684 non-null    float64
 2   AAAA        684 non-null    int32  
 3   COMUNA      684 non-null    int32  
 4   Longitud    684 non-null    float64
 5   Latitud     684 non-null    float64
 6   VICTIMA     684 non-null    object 
dtypes: float64(3), int32(2), object(2)
memory usage: 32.2+ KB


In [74]:
#Verifico que el analisis de los datos sean los correctos 
hechos21 = dfh[dfh['AAAA'] == 2021]['N_VICTIMAS'].sum()


In [75]:
print(hechos21)

96.0


### Salvo la transformacion a archivo parquet

In [83]:
df5.to_parquet(r'D:\0_Respaldo\0_Proyectos_2024\Henry_Labs\Lab2\Baires_Accidents\Data\ETL\siniestros_por_comuna.parquet')